In [1]:
import pickle
import graphlab as gl
import pandas as pd
from datetime import datetime, timedelta



[WARNING] graphlab.product_key: Unable to write current GraphLab Create license to /home/nawel/.graphlab/config. Ensure that this user account                         has write permission to /home/nawel/.graphlab/config to save the license for offline use.
[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1483481917.log


This non-commercial license of GraphLab Create for academic use is assigned to nawel.medjkoune@u-psud.fr and will expire on December 27, 2017.


[WARNING] graphlab.deploy._session: Unable to create session in specified location: '/home/nawel/.graphlab/artifacts'. Using: '/var/tmp/graphlab-nawel/54448/tmp_session_bef64323-6946-4df4-aeb1-fb98b8119f15'


In [2]:
data=pickle.load(open("gpCalles.obj","rb"))

In [3]:
dataTest=pickle.load(open("testCalls.obj","rb"))

In [ ]:
print data.head()

In [4]:
import numpy as np

class FeatureExtractorClf(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_df):
        pass
    
    def transform(self, X_df):

        XX = np.array([np.array(dd) for dd in X_df['ASS_ASSIGNMENT']])
        return XX

In [5]:
# coding: utf-8
# Author:  Daro HENG <daro.heng@u-psud.fr>, 
# Licence: BSD 3 clause


from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor                           
from sklearn.decomposition import PCA                                            
from sklearn.pipeline import Pipeline                                            
from sklearn.base import BaseEstimator    
from sklearn.neural_network import MLPRegressor
import numpy as np                                                               
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor


class Regressor(BaseEstimator):                                                  
    def __init__(self):                                                          
        self.n_components = 8                                                 
        self.n_estimators = 150                                              
        self.learning_rate = 0.2                                                
        self.reg = Pipeline([                                      
               #('pca', PCA(n_components=self.n_components)),        
                            
                ###### kernel PCA
                #('pca',  KernelPCA(n_components=21,kernel="rbf",remove_zero_eig=False)),


                ###### Gradient Boosting Regressor
                ('reg', GradientBoostingRegressor(max_depth=5, min_samples_leaf=10,n_estimators=self.n_estimators,learning_rate=self.learning_rate,random_state=42))

                
                ###### SVR
                #('reg',SVR(C=1.0, cache_size=200, coef0=0.0, degree=6, epsilon=0.2, gamma='auto',kernel='rbf', max_iter=5, shrinking=True, tol=0.001, verbose=False))
                
                ##### linear model
                #('reg',linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=5, normalize=False))
               
                ##### MLP Regressor
                #('reg',MLPRegressor(hidden_layer_sizes=(7,), activation='relu', solver='adam', alpha=0.0001, batch_size='auto',
                #learning_rate='constant', learning_rate_init=1, power_t=0.5, max_iter=1000, shuffle=True, random_state=42, 
                #tol=0.0001, verbose=False, warm_start=False, momentum=0.09, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1,
                #beta_1=0.9, beta_2=0.999, epsilon=1e-08))
                
               
                ##### Neighbors Regressor
                #('reg',RadiusNeighborsRegressor(radius=5.0))
                
                ##### PLS Regression
                #('reg', PLSRegression(copy=True, max_iter=500, n_components=2, scale=True,tol=1e-06))
                    
                ##### Decision Tree Regressor
                #('reg',DecisionTreeRegressor(max_depth=5))
                    
                ##### Random Forest Regressor
                #('reg',RandomForestRegressor(n_estimators=100))
                
                ##### Ada Boost Regressor avec Decision Tree Regressor
                #('reg', AdaBoostRegressor(DecisionTreeRegressor(max_depth=2),
                #        n_estimators=3000, random_state=42))
            ])                                                                   
                                                                                 
    def fit(self, X, y):
        self.reg.fit(X, y)

    def predict(self, X):
        return self.reg.predict(X)


In [7]:
# coding: utf-8
# Author:  Daro HENG <daro.heng@u-psud.fr>, 
# Licence: BSD 3 clause

import pickle
import graphlab as gl
import pandas as pd
from datetime import datetime, timedelta
import numpy as np                                                               
from utils import make_submission
from regressor import Regressor


labels = data['CSPL_RECEIVED_CALLS'].unique()

#soustraire la moyenne
mean_calls=data.groupby(['ASS_ASSIGNMENT', 'DAY_WE_DS','hour', 'minute']).mean().reset_index()
std_calls=data.groupby(['ASS_ASSIGNMENT', 'DAY_WE_DS','hour', 'minute']).std().reset_index()
data['MEAN_calls'] = pd.merge(data, mean_calls, how='left', on=['ASS_ASSIGNMENT', 'DAY_WE_DS','hour', 'minute'])['CSPL_RECEIVED_CALLS_y']
data['CSPL_RECEIVED_CALLS']=data['CSPL_RECEIVED_CALLS']-data['MEAN_calls']

#data['STD_calls'] = pd.merge(data, std_calls, how='left', on=['ASS_ASSIGNMENT', 'DAY_WE_DS','hour', 'minute'])['CSPL_RECEIVED_CALLS_y']
#data['CSPL_RECEIVED_CALLS']=data['CSPL_RECEIVED_CALLS']/data['STD_calls']

dataTest['MEAN_calls'] = pd.merge(dataTest, mean_calls, how='left', on=['ASS_ASSIGNMENT', 'DAY_WE_DS','hour', 'minute'])['CSPL_RECEIVED_CALLS']
#dataTest['STD_calls'] = pd.merge(dataTest, std_calls, how='left', on=['ASS_ASSIGNMENT', 'DAY_WE_DS','hour', 'minute'])['CSPL_RECEIVED_CALLS']

#### Load the X train and Y train
Y_train=data['CSPL_RECEIVED_CALLS']
X_train=data[  [  'DAY_WE_DS','cod_ASS_ASSIGNMENT','year','month','day','hour','minute']]
X_test=dataTest[[  'DAY_WE_DS','cod_ASS_ASSIGNMENT','year','month','day','hour','minute']]

Y_train=np.array(Y_train)
X_train=np.array(X_train)
X_test=np.array(X_test)

#### fit 
print "Fit ..."
reg=Regressor()
reg.fit(X_train, Y_train)




Fit ...


In [15]:
#### Prediction
print "Prediction ..."
Y_pred = reg.predict(X_test)
#rajouter le mean
Y_pred=(Y_pred)+dataTest['MEAN_calls']
Y_pred=Y_pred.apply(lambda x: (x,-x)[x<0])
#### write the submission
print "Write the submission ..."
make_submission1(dataTest,Y_pred)

print "End."

Prediction ...
Write the submission ...
End.


In [11]:
max(Y_pred)

969.11455172413775

-193.74520306513421

In [78]:
np.min(Y_pred)

0.0

In [71]:
pred=Y_pred+dataTest['MEAN_calls']

In [74]:
pred

0        0.000000e+00
1        0.000000e+00
2        0.000000e+00
3        0.000000e+00
4        7.103754e-03
5        2.550222e-02
6        0.000000e+00
7        3.882542e-02
8        2.473175e-01
9        1.882059e-01
10       1.871120e+01
11       0.000000e+00
12       2.971305e-01
13       1.872966e-02
14       7.177043e-01
15       2.415151e-02
16       1.930538e-01
17       6.107863e+00
18       0.000000e+00
19       0.000000e+00
20       9.372167e-03
21       0.000000e+00
22       0.000000e+00
23       0.000000e+00
24       0.000000e+00
25       1.180556e-02
26       3.871354e-01
27       3.127182e-02
28       1.326180e+01
29       2.846863e-02
             ...     
82879    1.398707e+00
82880    4.867203e-01
82881    0.000000e+00
82882    2.287429e+00
82883    5.692779e-02
82884    1.601338e+00
82885    1.549768e+01
82886    1.671986e+00
82887    7.126841e-01
82888    5.859311e+01
82889    6.302806e-02
82890    0.000000e+00
82891    0.000000e+00
82892    0.000000e+00
82893    0

In [14]:
from utils import *

def make_submission1(test, prediction, filename='submission.txt'):
    """
    Create a submission file, 
    test: test dataset
    prediction: predicted values
    """
    filename='submission-%s.txt'%datetime.now().strftime('%Y-%m-%d-%H:%M')

    with open(filename, 'w') as f:
        f.write('DATE\tASS_ASSIGNMENT\tprediction\n')
        submission_strings = test['DATE'] + '\t' + test['ASS_ASSIGNMENT'] + '\t'+ prediction.astype(str)
        for row in submission_strings:
            f.write(row + '\n') 
            


In [68]:
max(dataTest['MEAN_calls'])

300.48275862068897